# Import the required libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import joblib
from PIL import Image
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

/home/siddhaarth/anaconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/siddhaarth/anaconda3/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# AlexNet-based feature extractor

In [3]:
class AlexNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(AlexNetFeatureExtractor, self).__init__()
        self.features = models.alexnet(pretrained=pretrained).features
    
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return x

# SVM classifier

In [4]:
class SVMClassifier:
    def __init__(self):
        self.scaler = StandardScaler()
        self.classifier = svm.SVC(kernel='rbf')
    
    def train(self, X, y):
        X = self.scaler.fit_transform(X)
        self.classifier.fit(X, y)
    
    def predict(self, X):
        X = self.scaler.transform(X)
        return self.classifier.predict(X)

# Define the path to the dataset directory

In [5]:
dataset_path = "./dataset"

# Training and Testing process

In [6]:
# Load the dataset and apply transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Convert to RGB format
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
dataset = datasets.ImageFolder(dataset_path, transform=transform)

In [7]:
# Split the dataset into training and testing sets
train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

In [8]:
# Create the data loaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)

In [9]:
# Create an instance of the AlexNet feature extractor
feature_extractor = AlexNetFeatureExtractor()

/home/siddhaarth/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/siddhaarth/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
# Extract features from the training set
train_features = []
train_labels = []
for images, labels in train_loader:
    features = feature_extractor(images)
    train_features.extend(features.detach().numpy())
    train_labels.extend(labels.numpy())

In [11]:
# Train the SVM classifier
svm_classifier = SVMClassifier()
svm_classifier.train(train_features, train_labels)

In [12]:
# Extract features from the testing set
test_features = []
test_labels = []
for images, labels in test_loader:
    features = feature_extractor(images)
    test_features.extend(features.detach().numpy())
    test_labels.extend(labels.numpy())

In [13]:
# Make predictions using the trained classifier
predictions = svm_classifier.predict(test_features)

# Calculate Accuracy

In [14]:
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9503968253968254


In [15]:
# Save the trained classifier
joblib.dump(svm_classifier, "svm_classifier.pkl")

['svm_classifier.pkl']

# Prediction

In [16]:
# Load the trained classifier
svm_classifier = joblib.load("svm_classifier.pkl")

# List of input image paths
image_paths = ["characters/d.png", "characters/o.png", "characters/l.png", "characters/o.png", "characters/6.png", "characters/5.png", "characters/0.png"]

# List to store predicted class names
predicted_classes = []

# Process each input image
for image_path in image_paths:
    # Load and preprocess the input image
    input_image = Image.open(image_path)
    input_image = transform(input_image).unsqueeze(0)

    # Extract features from the input image
    input_features = feature_extractor(input_image).detach().numpy()

    # Make a prediction using the trained classifier
    prediction = svm_classifier.predict(input_features)

    # Convert the predicted label index to the corresponding class name
    class_names = dataset.classes
    predicted_class = class_names[prediction[0]]

    # Append the predicted class to the list
    predicted_classes.append(predicted_class)

# Print the predicted class names
med = "".join(predicted_classes)
print("Medicine: ",med)

Medicine:  DOLO650


# Performance measures

In [17]:
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix
confusion_mat = confusion_matrix(test_labels, predictions)
print("Confusion Matrix:")
print(confusion_mat)


Confusion Matrix:
[[ 9  0  0 ...  0  0  0]
 [ 0 11  0 ...  0  0  0]
 [ 0  0 15 ...  0  0  0]
 ...
 [ 0  0  0 ... 12  0  0]
 [ 0  0  0 ...  0 14  0]
 [ 0  0  0 ...  0  0 17]]


In [18]:
from sklearn.metrics import f1_score, recall_score, precision_score

# Calculate F1 score
f1 = f1_score(test_labels, predictions, average='weighted')
print("F1 Score:", f1)

F1 Score: 0.9604954111002498


In [19]:
# Calculate recall
recall = recall_score(test_labels, predictions, average='weighted')
print("Recall:", recall)

Recall: 0.9503968253968254


In [20]:
# Calculate precision
precision = precision_score(test_labels, predictions, average='weighted')
print("Precision:", precision)

Precision: 0.9830304928989139
